In [13]:
import pandas as pd
import numpy as np
import logging

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder

import blosum as bl

logging.basicConfig(level=logging.INFO)

np.random.seed(42)

In [14]:
NO_PEPTIDES_IN_CSV = 10
FOUR_DIGIT = True

ALLELE_SPECIFIC = False

project_data = pd.read_csv('../../DATA/allele-associated-dataset-multi-specific.csv', sep='\t')

project_data = project_data[project_data['Allele'].str.contains('A|B|C')]

#get the 188 alleles
alleles = project_data['Allele'].unique()
logging.info(f'amount of alleles in dataset: {len(alleles)}')
protein_lengths = project_data['Peptide'].str.len().unique()
protein_lengths.sort()
# Only take peptides with length 8 to 13
project_data = project_data[project_data['Peptide'].str.len().between(8, 13)]

project_data = project_data.groupby('Allele').filter(lambda x: len(x) >= NO_PEPTIDES_IN_CSV)
#make counter of how many peptides per allele
allele_counts = project_data['Allele'].value_counts()
print(allele_counts)

#Make input of RF same length by selecting first 4 and last 4 amino acids to catch anchor positions
def take_first_and_last_four(s):
        return s[:4] + s[-4:]

project_data['Peptide'] = project_data['Peptide'].apply(take_first_and_last_four)

train_sequences = project_data.copy()

if ALLELE_SPECIFIC:
     train_sequences.columns = ['label', 'sequence']
else:
    train_sequences.columns = ['label', 'sequence', 'source']
     
train_sequences

#drop all columns in train_sequences that dont start with a or b
#train_sequences = train_sequences[train_sequences['label'].str.startswith('A') | train_sequences['label'].str.startswith('B') ]

peptide_sequences = list(train_sequences['sequence'].values)
hla_alleles = list(train_sequences['label'].values)

#One Hot Encoding
'''onehot_encoder = OneHotEncoder(sparse=False)
X = onehot_encoder.fit_transform([[aa] for seq in peptide_sequences for aa in seq])
X = X.reshape(len(peptide_sequences), -1)'''

#BLOSUM
def encode_aa_blosum(aa: str):
    return list(bl.BLOSUM(62)[aa].values())
def encode_protein_blosum(p: str):
    return np.concatenate([encode_aa_blosum(aa) for aa in p])
project_data['Peptide'] = project_data['Peptide'].apply(encode_protein_blosum)

X = np.array(list(project_data['Peptide'].values))
X = X.reshape(len(peptide_sequences), -1)

X_train, X_test, y_train, y_test = train_test_split(X, hla_alleles, test_size=0.2, random_state=42, stratify=hla_alleles)

logging.info("we have {} training samples and {} test samples".format(X_train.shape[0], X_test.shape[0]))

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=None)

# Perform cross-validation with 5 folds
cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=5, n_jobs=None)

# Print cross-validation scores
logging.info(f"Cross-validation scores: {cv_scores}")
logging.info(f"Mean CV accuracy: {np.mean(cv_scores)}")

# Fit the classifier on the entire training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {np.round(accuracy, 2)}")

#f1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 score:", np.round(f1,2))



KeyboardInterrupt: 

In [4]:
'''##save 

NO_PEPTIDES_IN_CSV = 10
FOUR_DIGIT = True

ALLELE_SPECIFIC = True

project_data = pd.read_csv('all_binder_allele_specific.csv', sep='\t')

project_data = project_data[project_data['Allele'].str.contains('A|B|C')]

#get the 188 alleles
alleles = project_data['Allele'].unique()
logging.info(f'amount of alleles in dataset: {len(alleles)}')
protein_lengths = project_data['Peptide'].str.len().unique()
protein_lengths.sort()
# Only take peptides with length 8 to 13
project_data = project_data[project_data['Peptide'].str.len().between(8, 13)]

project_data = project_data.groupby('Allele').filter(lambda x: len(x) >= NO_PEPTIDES_IN_CSV)
#make counter of how many peptides per allele
allele_counts = project_data['Allele'].value_counts()
print(allele_counts)

#Make input of RF same length by selecting first 4 and last 4 amino acids to catch anchor positions
def take_first_and_last_four(s):
        return s[:4] + s[-4:]

project_data['Peptide'] = project_data['Peptide'].apply(take_first_and_last_four)
train_sequences = project_data.copy()

if ALLELE_SPECIFIC:
     train_sequences.columns = ['label', 'sequence']
else:
    train_sequences.columns = ['label', 'sequence', 'source']
     
train_sequences

#drop all columns in train_sequences that dont start with a or b
train_sequences = train_sequences[train_sequences['label'].str.startswith('A') | train_sequences['label'].str.startswith('B')]

peptide_sequences = list(train_sequences['sequence'].values)
hla_alleles = list(train_sequences['label'].values)

#Encoding
onehot_encoder = OneHotEncoder(sparse=False)
X = onehot_encoder.fit_transform([[aa] for seq in peptide_sequences for aa in seq])
X = X.reshape(len(peptide_sequences), -1)

X_train, X_test, y_train, y_test = train_test_split(X, hla_alleles, test_size=0.2, random_state=42, stratify=hla_alleles)

logging.info("we have {} training samples and {} test samples".format(X_train.shape[0], X_test.shape[0]))

rf_classifier_save = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=None)

# Perform cross-validation with 5 folds
cv_scores = cross_val_score(rf_classifier_save, X_train, y_train, cv=5, n_jobs=None)

# Print cross-validation scores
logging.info(f"Cross-validation scores: {cv_scores}")
logging.info(f"Mean CV accuracy: {np.mean(cv_scores)}")

# Fit the classifier on the entire training data
rf_classifier_save.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier_save.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {np.round(accuracy, 2)}")

#f1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 score:", np.round(f1,2))

'''

'##save \n\nNO_PEPTIDES_IN_CSV = 10\nFOUR_DIGIT = True\n\nALLELE_SPECIFIC = True\n\nproject_data = pd.read_csv(\'all_binder_allele_specific.csv\', sep=\'\t\')\n\nproject_data = project_data[project_data[\'Allele\'].str.contains(\'A|B|C\')]\n\n#get the 188 alleles\nalleles = project_data[\'Allele\'].unique()\nlogging.info(f\'amount of alleles in dataset: {len(alleles)}\')\nprotein_lengths = project_data[\'Peptide\'].str.len().unique()\nprotein_lengths.sort()\n# Only take peptides with length 8 to 13\nproject_data = project_data[project_data[\'Peptide\'].str.len().between(8, 13)]\n\nproject_data = project_data.groupby(\'Allele\').filter(lambda x: len(x) >= NO_PEPTIDES_IN_CSV)\n#make counter of how many peptides per allele\nallele_counts = project_data[\'Allele\'].value_counts()\nprint(allele_counts)\n\n#Make input of RF same length by selecting first 4 and last 4 amino acids to catch anchor positions\ndef take_first_and_last_four(s):\n        return s[:4] + s[-4:]\n\nproject_data[\'Pepti

# Validation on TueDB (with 4-digit)

In [5]:
def predict_peptide_list(peptides):
    peptides = [p[:4] + p[-4:] for p in peptides] 

    '''ex_encoded = onehot_encoder.transform([[aa] for seq in peptides for aa in seq])
    ex_encoded_reshaped = ex_encoded.reshape(len(peptides), -1)'''
    ex_encoded = np.array([encode_protein_blosum(p) for p in peptides])
    ex_encoded_reshaped = ex_encoded.reshape(len(peptides), -1)

    prediction = rf_classifier.predict(ex_encoded_reshaped)

    return prediction

In [6]:
PATH_TO_TUEDB = '../db_dump_311023_cleaned.tsv'

tuedb_df = pd.read_csv(PATH_TO_TUEDB, sep='\t',index_col=0)

tdb_df = tuedb_df.copy()

#get all unique alleles
alleles = tdb_df['all_hla_alleles_donor'].explode().unique()
donors = tdb_df['donor_code'].unique()
tdb_df = tdb_df.sort_values(by=['donor_code'])
group1_df = tdb_df.groupby('donor_code')['peptide_sequence'].apply(list).reset_index()
group1_df['alleles'] = tdb_df.groupby('donor_code')['all_hla_alleles_donor'].apply(list).reset_index()['all_hla_alleles_donor']
group1_df['alleles'] = group1_df['alleles'].apply(lambda x: x[0])
display(group1_df.head())

,donor_code,peptide_sequence,alleles
0,04-001,"[TTDLFGRDLSY, AYLEAHETF, NRFQIATV, TAASRLVTL, ...","['A0101', 'A2402', 'B0801', 'B1402', 'C0701', ..."
1,1003,"[DAVTAFESI, PIDGNFFTY, FLSFMNTEL, YTWEEVFRV, T...","['A0101', 'A0201', 'B5101', 'B5701', 'C0401', ..."
2,1008,"[SLFEEMLQV, TLIDLPGITKV, VVYEGQLISI, AEFKEAFQL...","['A0101', 'A0201', 'B0801', 'B4001', 'C0304', ..."
3,1010,"[ALWSLPLYL, FLLPILSQI, DAYVILKTV, IYEPNFIFF, L...","['A0201', 'A2402', 'B5001', 'B5101', 'C0102', ..."
4,1012,"[TLLPLRVFL, VLWDRTFSLF, SLLDIIEKV, SRLPVLLLL, ...","['A0201', 'A0301', 'B0702', 'B4402', 'C0501', ..."


In [7]:
#import counter
from collections import Counter

predictions = {}

for index, row in group1_df.iterrows():
    peptides = row['peptide_sequence']
    alleles = row['alleles']
    prediction = predict_peptide_list(peptides)
    predictions[row['donor_code']] = {'pred': dict(Counter(prediction)), 'tuedb': alleles}

with open('predictions.txt', 'w') as file:
    file.write(str(predictions))
    file.close()

In [8]:
accuracy = []

for donor in predictions:

    tuedb = eval(predictions[donor]['tuedb'])

    if len(tuedb) > 4 and len(tuedb) < 7:
        pred = predictions[donor]['pred']

        pred_a = {k: v for k, v in pred.items() if k.startswith('A')}
        pred_b = {k: v for k, v in pred.items() if k.startswith('B')}
        pred_c = {k: v for k, v in pred.items() if k.startswith('C')}

        #pick the top 2 keys that have the highest value
        pred_a = list(dict(sorted(pred_a.items(), key=lambda item: item[1], reverse=True)[:2]).keys())
        pred_b = list(dict(sorted(pred_b.items(), key=lambda item: item[1], reverse=True)[:2]).keys())
        pred_c = list(dict(sorted(pred_c.items(), key=lambda item: item[1], reverse=True)[:2]).keys())

        prediction  = pred_a + pred_b + pred_c

        matching_count = 0
        for donor_allele in tuedb:
            if donor_allele in prediction:
                matching_count += 1

        perc_matching = matching_count/len(tuedb)

        accuracy = accuracy + [perc_matching]

print("Accuracy:", np.round(np.mean(accuracy),2))

Accuracy: 0.66
